# <center>Tobacco Documents Classificationc</center>

# <center>Mathieu VANDECASTEELE - MASTER SID 2018-2019</center>

## <center>Université de Rouen</center>

Joint à ce rapport : scripts Python

Ce document a pour but de rapporter l'étude de la classification des types des différents documents émis lors des procès contre les industries du tabac aux États-Unis.

Plan de l'étude :
* Description statistique et analyse du jeu de données
* Découpe / Préparation / Pré-Traitement des données
* Apprentissages de classifieurs
* Conclusion, dernières analyses et Pistes d'amélioration

# 1. Analyse des données

L'État américain a collecté plus de 14 millions de documents, ici nous disposons d'un échantillon de 3482 éléments précisemment. 
Nous devons établir un classifieur de documents, nous avons 10 types différents à différenciers : Email, Advertisement, Form, Letter, etc.
Première constatation dans un premier temps : 3482 données ce n'est pas beaucoup, cela pourra plus tard poser des problèmes, surtout pour une classification multi-classes.


Nous disposons comme données d'un .csv avec l'image_path et le label associé pour chacune. Nous disposons également des textes de ces images obtenus par OCR.

In [3]:
# Premiers Import nécessaires et Import des données.
import pandas as pd
classes = pd.read_csv('tobacco-lab/data/Tobacco3482.csv', sep = ",")

### Répartition des données

Analysons la répartition des données avec quelques statistiques descriptives. Tout d'abord piochons 10 données au hasard pour établir une première observation :


In [14]:
classes.sample(10)

,img_path,label
1045,Form/2070934900.jpg,Form
323,Email/2074848927.jpg,Email
438,Email/2078881416.jpg,Email
2554,News/2047564059.jpg,News
2435,Memo/tob03508.13.jpg,Memo
272,Email/2071862304d_2305.jpg,Email
301,Email/2072948072a.jpg,Email
219,Advertisement/93341527.jpg,Advertisement
2495,News/10032025.jpg,News
3167,Resume/50510506-0507.jpg,Resume


Il semble y avoir plus d'emails que d'autres types de documents dans notre jeu de données. Avons-nous une sur-représentation d'une classe particulière ?